## StyleGAN2 - ADA
- 4번 결함 학습 및 fakeimage생성

- 참고1 : StyleGAN2-ADA — Official PyTorch implementation
https://github.com/NVlabs/stylegan2-ada-pytorch
- 참고2 : 강콩콩 https://velog.io/@gtpgg1013/GAN-%EB%95%8C%EB%AC%B8%EC%9D%B4%EC%95%BC-Generative-Models-%ED%95%99%EC%8A%B5%EA%B8%B0-1

In [ ]:
!python -V


import torch

print(torch.__version__)

Python 3.10.12
2.1.0+cu118


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Oct 31 04:05:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# Preparing
1. Gdrive Mount
2. git clone

In [ ]:
#!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 torchaudio==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu116
#!pip install torch==1.7.1+cu116 torchvision==0.8.2+cu116 torchaudio==0.7.2 --extra-index-url https://download.pytorch.org/whl/cu116
#!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2
#!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2

!pip install ninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.2 MB/s eta 0:00:00


# Train

In [ ]:
# model_save_dir : google drive에 model 저장 path
# train_data_dir : 학습에 사용할 data path (directory, not zip)
model_save_dir = '/content/drive/MyDrive/stylegan/stylegan2-ada-pytorch-main/stylegan_model'# '/content/drive/MyDrive/GAN_때문이야/model/20221005'
train_data_dir = '/content/drive/MyDrive/stylegan/stylegan2-ada-pytorch-main/defect4_cropped' #'/content/drive/MyDrive/GAN_때문이야/dataset/4'


In [ ]:
ls

calc_metrics.py   defect4_cropped.zip  docker_run.sh  LICENSE.txt   README.md        torch_utils/
datasets/         defect4.zip          docs/          metrics/      stylegan         train_dataset/
dataset_tool.py   dnnlib/              generate.py    projector.py  stylegan_model/  training/
defect4_cropped/  Dockerfile           legacy.py      __pycache__/  style_mixing.py  train.py


In [ ]:
import os

os.chdir('/content/drive/MyDrive/stylegan/stylegan2-ada-pytorch-main')
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1YAxQ9GyveLoNgPvh21dJgpE9ZwXna24h/stylegan/stylegan2-ada-pytorch-main'

In [ ]:
import os

# Path
train_data_dir = '/content/drive/MyDrive/stylegan/stylegan2-ada-pytorch-main/defect4_cropped'
os.environ['train_data_dir'] = train_data_dir

# zip
!python dataset_tool.py --source=$train_data_dir --dest=/content/drive/MyDrive/stylegan/stylegan2-ada-pytorch-main/defect4.zip

In [ ]:
os.environ['model_save_dir'] = model_save_dir

!python train.py --outdir=$model_save_dir --data=/content/drive/MyDrive/stylegan/stylegan2-ada-pytorch-main/defect4.zip --cfg=auto --gpus=1 --kimg=100 --snap=25 --workers=2

# Inference

In [ ]:
# inference test
!python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"...
Generating image for seed 85 (0/4) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 265 (1/4) ...
Generating image for seed 297 (2/4) ...
Generating image for seed 849 (3/4) ...


In [ ]:
import os

# str(list(range(10000)))[1:-1].replace(' ', '')

os.environ['sample_nums'] = str(list(range(100)))[1:-1].replace(' ', '')

In [ ]:
# 참고 : https://velog.io/@gtpgg1013/GAN-%EB%95%8C%EB%AC%B8%EC%9D%B4%EC%95%BC-Generative-Models-%ED%95%99%EC%8A%B5%EA%B8%B0-1
# 수정 필요사항
# generate.py : line 121
# 		PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png') # 주석처리

# 아래 내용으로 수정 (indent 주의)
        if img.shape[-1] == 3:
            PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png')
        else:
            PIL.Image.fromarray(np.squeeze(img[0][:-2].cpu().numpy()), 'L').save(f'{outdir}/seed{seed:04d}.png')

In [ ]:
# random seed 1~10000 까지 data generation
# trunc는 truncation trick의 계수, 0~1 : 0으로 지정하면 모두 같은 얼굴 생성 / 1로 지정하면 모두 다른 이미지 생성

!python generate.py --outdir=/content/drive/MyDrive/stylegan/stylegan2-ada-pytorch-main/stylegan_model/00003-defect4-auto1-kimg100/fakeimages --trunc=0.9 --seeds=$sample_nums --network=/content/drive/MyDrive/stylegan/stylegan2-ada-pytorch-main/stylegan_model/00003-defect4-auto1-kimg100/network-snapshot-000100.pkl

Loading networks from "/content/drive/MyDrive/stylegan/stylegan2-ada-pytorch-main/stylegan_model/00003-defect4-auto1-kimg100/network-snapshot-000100.pkl"...
Generating image for seed 0 (0/100) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 1 (1/100) ...
Generating image for seed 2 (2/100) ...
Generating image for seed 3 (3/100) ...
Generating image for seed 4 (4/100) ...
Generating image for seed 5 (5/100) ...
Generating image for seed 6 (6/100) ...
Generating image for seed 7 (7/100) ...
Generating image for seed 8 (8/100) ...
Generating image for seed 9 (9/100) ...
Generating image for seed 10 (10/100) ...
Generating image for seed 11 (11/100) ...
Generating image for seed 12 (12/100) ...
Generating image for seed 13 (13/100) ...
Generating image for seed 14 (14/100) ...
Generating image for seed 15 (15/100) ...
Generating image for seed 16 (16/100) ...
Generating image for seed 17 (17/100) ...
